<a href="https://colab.research.google.com/github/GreihMurray/NLP-3/blob/Super_Murray/supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Reshape
from tensorflow.keras.optimizers import SGD
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
import joblib
from sklearn.metrics import accuracy_score
import pickle
import nltk
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


Original

In [ ]:
def read_file_to_sents():
    all_data = []
    with open("/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/train.tsv", encoding="utf-8") as file:
        f = csv.reader(file, delimiter="\t")
        for line in tqdm(f, desc="Reading data..."):
            word = line[0]
            graphemes = line[1].split('-')

            cur_word = []

            for i in range(0, len(graphemes)):
                for j in range(0, len(graphemes[i])):
                    if j == 0:
                        cur_word.append((graphemes[i][j], 'B'))
                    else:
                        cur_word.append((graphemes[i][j], 'I'))

            all_data.append(cur_word)

    return all_data

From Challenge 2 https://github.com/GreihMurray/NLP-2

In [ ]:
def pad(data):
  vocab = list(set([w for sent in data for (w,t) in sent]))
  vocab.append('<PAD>')
  tags = list(set([t for sent in data for (w,t) in sent]))
  tags.append('<PAD>')

  return vocab, tags

Combination of code from Challenge 2 (https://github.com/GreihMurray/NLP-2) and custom

In [ ]:
def encode(vocab, tags, data, load=False):
  max_len = max([len(i) for i in data])

  word2index = {}
  tag2index = {}

  if load is False:
      word2index = {w: i for i, w in enumerate(vocab)}
      tag2index = {t: i for i, t in enumerate(tags)}
  else:
      with open("/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/word2index.json") as infile:
          word2index = json.load(infile)  

      with open("/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/tag2index.json") as outfile:
          tag2index = json.load(outfile)

  onehot = [[word2index[w[0]] for w in s] for s in data]
  X = pad_sequences(maxlen=max_len, sequences=onehot, padding="post", value=len(vocab)-1)  

  onehot_y = [[tag2index[w[1]] for w in s] for s in data]
  y = pad_sequences(maxlen=max_len, sequences=onehot_y, padding="post", value=tag2index["<PAD>"])
  y = to_categorical(y, num_classes=len(tags))

  # Used for saving word2index and tag2index in order to encode additional data in the same manner
  # Currently commented out due to issues with loading model
  with open("/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/word2index.json", "w") as outfile:
    json.dump(word2index, outfile)

  with open("/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/tag2index.json", "w") as outfile:
    json.dump(tag2index, outfile)

  return X, y, max_len

Code from Challenge 2 & Dr. Scannell (https://github.com/GreihMurray/NLP-2, 

In [ ]:
def seq_model(data):
  #Original
    vocab, tags = pad(data)

    x, y, max_len = encode(vocab, tags, data)

    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
  
  
  # Dr. Scannell
    model = Sequential()
    model.add(Embedding(input_dim=len(vocab), output_dim=50, input_length=max_len))
    model.add(Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.05)))
    model.add(TimeDistributed(Dense(len(tags), activation="softmax")))
    model.compile(optimizer="adam", loss="poisson", metrics=["accuracy"])
  # From https://towardsdatascience.com/hyperparameter-tuning-with-kerastuner-and-tensorflow-c4a4d690b31a
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

    print("[INFO] training network...")
    sgd = SGD(0.05)
    history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_split=0.15, verbose=1, callbacks=stop_early)

    return model, X_test, y_test

Original

In [ ]:
def eval_model(model, x_test, y_test):
    eval = model.evaluate(x_test, y_test)
    print(eval)

Original

In [ ]:
def supervised():
    data = read_file_to_sents()

    model, x_test, y_test = seq_model(data)

    eval_model(model, x_test, y_test) # Eval sequential model

    model.save('/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/adamPoisson32_seq_model')

    new_model = load_model('/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/adamPoisson32_seq_model')

    eval_model(new_model, x_test, y_test)

Original

In [24]:
def load_and_eval_model():
    data = read_file_to_sents()

    vocab, tags = pad(data)

    x, y, max_len = encode(vocab, tags, data, load=True)

    new_model = load_model('/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/adamPoisson32_seq_model')

    evals = new_model.evaluate(x, y)

    print('Default Accuracy: ', (evals[1] * 100))

    custom_eval(new_model, x, y)

Original

In [ ]:
def undo_encode(x, y):
    all_data = []
    all_words = []
    all_tags = []

    word2index = {}
    tag2index = {}

    with open("/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/word2index.json") as infile:
          word2index = json.load(infile)  

    with open("/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/tag2index.json") as outfile:
          tag2index = json.load(outfile)

    for word in x:
        cur_word = []
        for letter in word:
            if letter == 51:
                break
            true_letter = list(word2index.keys())[list(word2index.values()).index(letter)]
            cur_word.append(true_letter)

        all_words.append(''.join(cur_word))

    for tags in y:
        cur_tags = []
        for tag in tags:
            if tag[0] == 1:
                cur_tags.append('I')
            elif tag[1] == 1:
                cur_tags.append('B')
            else:
                continue

        all_tags.append(''.join(cur_tags))

    for i in range(0, len(all_words)):
        all_data.append((all_words[i], all_tags[i]))

    return all_data

Original

In [ ]:
def undo_encode_y(y):
    all_tags = []

    word2index = {}
    tag2index = {}

    with open("/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/word2index.json") as infile:
          word2index = json.load(infile)  

    with open("/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/tag2index.json") as outfile:
          tag2index = json.load(outfile)

    for tags in y:
        cur_tags = []
        for tag in tags:
            if round(tag[0]) == 1:
                cur_tags.append('I')
            elif round(tag[1]) == 1:
                cur_tags.append('B')
            else:
                continue

        all_tags.append(''.join(cur_tags))

    return all_tags

Original

In [22]:
def calc_accuracy(preds, y_test):
    total_right = 0

    for i in range(0, len(preds)):
        if preds[i] == y_test[i]:
            total_right += 1

    accuracy = 100 * (total_right/len(preds))

    return accuracy

Original

In [ ]:
def calc_precision(preds, y_test):
    true_pos = 0
    false_pos = 0

    for i in range(0, len(preds)):
        for j in range(0, len(preds[i])):
            if preds[i][j] == 'I' and y_test[i][j] == 'I':
                true_pos += 1
            if preds[i][j] == 'I' and y_test[i][j] == 'B':
                false_pos += 1

    if (true_pos + false_pos) == 0:
        return 0.01

    precision = 100 * (true_pos / (true_pos + false_pos))

    return precision

Original

In [ ]:
def calc_recall(preds, y_test):
    true_pos = 0
    false_neg = 0

    for i in range(0, len(preds)):
        for j in range(0, len(preds[i])):
            if preds[i][j] == 'I' and y_test[i][j] == 'I':
                true_pos += 1
            if preds[i][j] == 'B' and y_test[i][j] == 'I':
                false_neg += 1

    if true_pos + false_neg == 0:
        return 0
        
    recall = 100 * (true_pos / (true_pos + false_neg))

    return recall

Original

In [23]:
def custom_eval(model, x, y):
    preds = model.predict(x)

    preds = undo_encode_y(preds)
    y_clean = undo_encode_y(y)

    acc = calc_accuracy(preds, y_clean)

    print("Custom calculated Accuracy: ", acc)

    prec = calc_precision(preds, y_clean)

    print("Precision: ", prec)

    recall = calc_recall(preds, y_clean)

    print("Recall: ", recall)

    fscore = (2 * (prec * recall)) / (prec + recall)

    print("Fscore: ", fscore)

Original

In [ ]:
def to_graphemes(data):
    graph_data = []

    for word_pair in data:
        word = word_pair[0]
        grap = word_pair[1]

        cur_word = []

        for i in range(0, len(word)):
            if i == (len(word) - 1):
                cur_word.append(word[i])

            else:
                if grap[i+1] == 'I':
                    cur_word.append(word[i])
                else:
                    cur_word.append(word[i] + '-')

        graph_data.append((word, ''.join(cur_word)))

    return graph_data


Original

In [25]:
load_and_eval_model()

Reading data...: 12812it [00:00, 135481.26it/s]


401/401 [==============================] - 6s 13ms/step - loss: 0.3335 - accuracy: 0.9998
Default Accuracy:  99.9809205532074
Custom calculated Accuracy:  99.66437714642524
Precision:  99.93399339933994
Recall:  99.50052576235542
Fscore:  99.71678851346901


Original

In [ ]:
supervised()

Reading data...: 12812it [00:00, 133664.87it/s]


[INFO] training network...
Epoch 1/50


KeyboardInterrupt: ignored